# Introduction to comparative analysis of cell-cell communication and spatial analysis

Ivan G. Costa$^1$, James Nagai$^1$, Mayra Ruiz$^1$, Giullia Cesaro$^2$ 

*Tool:* ISCHIA[1]

Github: [ati-lz/ISCHIA](https://github.com/ati-lz/ISCHIA)

$^1$ Institute for Computational Genomics, RWTH Aachen University, Pauwelstr 19, Aachen, 52074, NRW, Germany.

$^2$ Department of Information Engineering, University of Padova, Street,City, 100190, State, Country.

_References_

[1]Lafzi, Atefeh, Costanza Borrelli, Simona Baghai Sain, Karsten Bach, Jonas A. Kretz, Kristina Handler, Daniel Regan-Komito, Xenia Ficht, Andreas Frei, and Andreas Moor. "Identifying spatial co-occurrence in healthy and inflamed tissues (ISCHIA)." Molecular Systems Biology 20, no. 2 (2024): 98-119.

### Step1: We will load all libraries used for the analysis

In [ ]:
# Loading required packages
library(ISCHIA)
library(robustbase)
library(data.table)
library(ggplot2)
library(Seurat)
library(dplyr)
library(MuDataSeurat)
library(hdf5r)
set.seed(42)
library(tidyverse)

### Step2: Reading and Visualizing the data

In [ ]:
adata <- readRDS("/Tutorial_ISMBECCB2025/data_vt3/second/ischia/FZ_P14.rds")
adata@assays$Spatial<-adata@assays$RNA

In [ ]:
options(repr.plot.width=16,repr.plot.height=7)
p1<-DimPlot(adata,reduction = 'spatial',group.by = "cell_type_original",pt.size = 2)
p2<-DimPlot(adata,reduction = 'spatial',pt.size = 2)
p1+p2

### Step3: Reading the deconvolution results

In [ ]:
deconv.mat <- as.matrix(adata@meta.data[,4:14])
head(deconv.mat)

### Step4: Selecting the ideal number of clusters

In [ ]:
options(repr.plot.width=7,repr.plot.height=6)

Composition.cluster.k(deconv.mat, 20)

### Step5: Composition clustering of the deconvoluted spatial spots

In [ ]:
options(repr.plot.width=7,repr.plot.height=6)
adata <- Composition.cluster(adata, deconv.mat, 6)
table(adata$CompositionCluster_CC)

DimPlot(adata, group.by = c("CompositionCluster_CC"),reduction = 'spatial',pt.size = 2) +
    scale_fill_manual(values = c("cyan", "orange", "purple","green","yellow","blue", "red","black"))

### Step6: Contribuition of the cell types per cluster

In [ ]:
deconv.mat |>
    as_tibble () |>
    mutate(cc = adata$CompositionCluster_CC) |>
    filter(cc == "CC3") |>
    reshape2::melt() |>
    ggplot(aes(x=variable,y=value,fill=variable))+
          geom_boxplot()+theme_minimal()

In [ ]:
deconv.mat |>
    as_tibble () |>
    mutate(cc = adata$CompositionCluster_CC) |>
    filter(cc == "CC2") |>
    reshape2::melt() |>
    ggplot(aes(x=variable,y=value,fill=variable))+
          geom_boxplot()+theme_minimal()

In [ ]:
adata$orig.ident = adata$orig_ident

### Step7: Co-localization patterns per Compositional Cluster

In [ ]:
options(repr.plot.width=15)
CC3 <- spatial.celltype.cooccurence(spatial.object=adata,
                                                     deconv.prob.mat=deconv.mat,
                                                     COI="CC3", prob.th= 0.05, 
                                                     Condition=unique(adata$orig_ident))

p1<-plot.celltype.cooccurence(CC3)
CC2 <- spatial.celltype.cooccurence(spatial.object=adata,
                                                     deconv.prob.mat=deconv.mat,
                                                     COI="CC2", prob.th= 0.05, 
                                                     Condition=unique(adata$orig_ident))

p2<-plot.celltype.cooccurence(CC2)
p1+p2

### Step7: Compositional Cluster related Cell Cell Communication Patterns

In [ ]:
lr_network = readRDS(url("https://zenodo.org/record/3260758/files/lr_network.rds"))
all.LR.network <- cbind(lr_network[,c("from","to")], LR_Pairs=paste(lr_network$from, lr_network$to, sep = "_"))
all.LR.network.exp <- all.LR.network[which(all.LR.network$from %in% rownames(adata) & all.LR.network$to %in% rownames(adata)),]

# To reduce the computation time for this example, we randomly sample from the whole dataset of LR interactions
all.LR.network.exp <- sample_n(all.LR.network.exp,200)
all.LR.genes <- unique(c(all.LR.network.exp$from, all.LR.network.exp$to))
all.LR.genes.comm <- intersect(all.LR.genes, rownames(adata))
LR.pairs <- all.LR.network.exp$LR_Pairs
LR.pairs.AllCombos <- combn(all.LR.genes.comm, 2, paste0, collapse = "_")

#### This step is time consuming, therefore, we will load the results from the functions below

In [ ]:
# cc3LRs <- Enriched.LRs(adata, c("CC3"), 
#                                  unique(adata$orig.ident), 
#                                  all.LR.genes.comm, LR.pairs, 1, 0.2)
# cc2LRs <- Enriched.LRs(adata, c("CC2"), 
#                                  unique(adata$orig.ident), 
#                                  all.LR.genes.comm, LR.pairs, 1, 0.2)
# saveRDS(cc3LRs,"cc3LRs.RDS")
# saveRDS(cc2LRs,"cc2LRs.RDS")

In [ ]:
cc3LRs <- readRDS("/Tutorial_ISMBECCB2025/data_vt3/second/ischia/cc3LRs.RDS")
cc2LRs <- readRDS("/Tutorial_ISMBECCB2025/data_vt3/second/ischia/cc2LRs.RDS")

In [ ]:
cc33vscc2 <- Diff.cooc.LRs(cc3LRs, cc2LRs, 0.05, 0.1)


In [ ]:
ChordPlot.Enriched.LRs(cc2LRs$COI.enrcihed.LRs[1:20,])

In [ ]:
SankeyPlot.Diff.LRs(cc2LRs$COI.enrcihed.LRs, cc3LRs$COI.enrcihed.LRs)

In [ ]:
sessionInfo()